In [7]:
import os

spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Module_16_Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [9]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...|2015-08-31 00:00:00|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed| 

In [10]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
vine_reviews_clean_df = df.dropna()

In [11]:
# Create the vine_table DataFrame
vine_df = vine_reviews_clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

Filter Data


In [12]:
# Filter for 20+ total votes
vine_filtered_votes_df = vine_df.filter("total_votes>=20")
vine_filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
| RUNB838XKMB59|          1|            8|         23|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
|R2WFUSIPJ3X0FR|          1|            9|         53|   N|                N|
| RONWXF0J85CRR|          5|           42|         50|   N|     

In [13]:
#  Filter for 20+ total votes and at least 50% helpful
vine_helpful_by_total_df = vine_filtered_votes_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_by_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [14]:
# Filter for paid reviews and at least 50% helpful
vine_paid_helpful_df = vine_helpful_by_total_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PBQ665MBL8CP|          3|           66|         72|   Y|                N|
|R3EMD30K7MNOZ0|          2|           20|         25|   Y|                N|
|R2GF3ZLO2B0Y71|          5|          236|        243|   Y|                N|
|R1F8T2Z7U3N6HL|          3|           11|         20|   Y|                N|
| R5QTOT1GVR8RG|          4|           16|         20|   Y|                N|
|R1KAQEESF0NUGV|          4|           27|         29|   Y|                N|
|R1621B9AMVWI25|          2|           27|         29|   Y|                N|
|R3FVGQVNBAZL8E|          3|           37|         44|   Y|                N|
|R26ATTA765WHVX|          5|           21|         23|   Y|                N|
| RMZYPFKES1ZK7|          4|           29|         33|   Y|     

In [15]:
# Filter for unpaid reviews and at least 50% helpful
vine_unpaid_helpful_df = vine_helpful_by_total_df.filter("vine == 'N'")
vine_unpaid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

Totals


In [16]:
# Total number of reviews that were helpful
total_helpful_reviews = vine_helpful_by_total_df.count()
total_helpful_reviews

189404

In [17]:
# Total number of vine reviews
vine_paid_helpful = vine_paid_helpful_df.count()
vine_paid_helpful

219

In [24]:
# Total number of non-Vine reviews
vine_unpaid_helpful = vine_unpaid_helpful_df.count()
vine_unpaid_helpful

189185

In [25]:
# 5 Star helpful reviews total
five_star_reviews_df = vine_helpful_by_total_df.filter("star_rating == 5")
five_star_reviews = five_star_reviews_df.count()
five_star_reviews

98713

In [26]:
# 5 Star helpful reviews that were paid total 
five_star_reviews_paid_df = vine_paid_helpful_df.filter("star_rating == 5")
five_star_reviews_paid = five_star_reviews_paid_df.count()
five_star_reviews_paid

74

In [27]:
# 5 Star helpful reviews that were not paid total  
five_star_reviews_unpaid_df = vine_unpaid_helpful_df.filter("star_rating == 5")
five_star_reviews_unpaid = five_star_reviews_unpaid_df.count()
five_star_reviews_unpaid

98639

In [28]:
# % 5 Star Reviews from those helpful that were paid for
percentage_five_star_reviews_paid = (five_star_reviews_paid/vine_paid_helpful)*100
round(percentage_five_star_reviews_paid, 3)

33.79

In [29]:
# % 5 Star Reviews from those helpful that were not paid for
percentage_five_star_reviews_unpaid = (five_star_reviews_unpaid/vine_unpaid_helpful)*100
round(percentage_five_star_reviews_unpaid, 3)

52.139